In [194]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import plotly.express as px
import gensim
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Import all the required libraries
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
#import stopwords and text processing libraries
import warnings
warnings.filterwarnings('ignore')

In [195]:
from wordcloud import WordCloud

In [196]:
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [197]:
#%%time
#final = pd.DataFrame()
#for j in range(1,360):
#    url = f'https://www.airlinequality.com/airline-reviews/british-airways/page/{j}/'
#    d = requests.get(url).text
#    soup = BeautifulSoup(d, 'lxml')
#    company = soup.find_all('article',class_='comp_media-review-rated')
#
#    title = []
#    reviews = []
#    date = []
#    place = []
#
#    for i in company:
#        title.append(i.find('h2', class_='text_header').text.strip())
#    ###---------------------------------------------------------------###
#        reviews.append(i.find('div',class_="text_content").text.strip())
#    ###---------------------------------------------------------------###
#        date.append(i.find('time').text.strip())
#    ###---------------------------------------------------------------###
#        text = i.find('h3',class_='text_sub_header').text
#        match = re.search(r'\((.*?)\)', text)
#        if match:
#            plac = match.group(1)
#            place.append(plac)
#    ###---------------------------------------------------------------###
#
#    df = pd.DataFrame({'title':title, 'reviews':reviews, 'date':date,'place':place})
#    ###---------------------------------------------------------------###
#
#    final = final.append(df)

In [198]:
df = pd.read_csv('/content/briaircusrev.csv')

In [199]:
df.head()

,Unnamed: 0,title,reviews,date,place
0,0,"""no boarding drinks provided""",✅ Trip Verified | First time flying with BA b...,25th June 2023,United Kingdom
1,1,"""WiFi didn't work""",Not Verified | You can buy sandwiches and cris...,24th June 2023,United Kingdom
2,2,"""stick with economy""",✅ Trip Verified | This is a two-for-one revie...,24th June 2023,United Kingdom
3,3,"""Communication is terrible""",✅ Trip Verified | Absolutely horrible airline....,23rd June 2023,United States
4,4,"""delays and cancellations""",✅ Trip Verified | Having experienced delays a...,22nd June 2023,United States


In [200]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [201]:
df.head()

,title,reviews,date,place
0,"""no boarding drinks provided""",✅ Trip Verified | First time flying with BA b...,25th June 2023,United Kingdom
1,"""WiFi didn't work""",Not Verified | You can buy sandwiches and cris...,24th June 2023,United Kingdom
2,"""stick with economy""",✅ Trip Verified | This is a two-for-one revie...,24th June 2023,United Kingdom
3,"""Communication is terrible""",✅ Trip Verified | Absolutely horrible airline....,23rd June 2023,United States
4,"""delays and cancellations""",✅ Trip Verified | Having experienced delays a...,22nd June 2023,United States


In [202]:
#df.set_index('date',inplace=True)
#df.head()

In [203]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    2930 non-null   object
 1   reviews  2930 non-null   object
 2   date     2930 non-null   object
 3   place    2930 non-null   object
dtypes: object(4)
memory usage: 91.7+ KB


In [204]:
df.isnull().sum()

title      0
reviews    0
date       0
place      0
dtype: int64

In [205]:
df['place'].value_counts()

United Kingdom           1826
United States             332
Australia                 113
Canada                     92
Germany                    54
                         ... 
Costa Rica                  1
Cayman Islands              1
Panama                      1
Saint Kitts and Nevis       1
Ukraine                     1
Name: place, Length: 67, dtype: int64

In [206]:
df['date'] = pd.to_datetime(df['date'])

In [207]:
df.head()

,title,reviews,date,place
0,"""no boarding drinks provided""",✅ Trip Verified | First time flying with BA b...,2023-06-25,United Kingdom
1,"""WiFi didn't work""",Not Verified | You can buy sandwiches and cris...,2023-06-24,United Kingdom
2,"""stick with economy""",✅ Trip Verified | This is a two-for-one revie...,2023-06-24,United Kingdom
3,"""Communication is terrible""",✅ Trip Verified | Absolutely horrible airline....,2023-06-23,United States
4,"""delays and cancellations""",✅ Trip Verified | Having experienced delays a...,2023-06-22,United States


In [208]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [209]:
df.head()

,title,reviews,date,place,year,month,day
0,"""no boarding drinks provided""",✅ Trip Verified | First time flying with BA b...,2023-06-25,United Kingdom,2023,6,25
1,"""WiFi didn't work""",Not Verified | You can buy sandwiches and cris...,2023-06-24,United Kingdom,2023,6,24
2,"""stick with economy""",✅ Trip Verified | This is a two-for-one revie...,2023-06-24,United Kingdom,2023,6,24
3,"""Communication is terrible""",✅ Trip Verified | Absolutely horrible airline....,2023-06-23,United States,2023,6,23
4,"""delays and cancellations""",✅ Trip Verified | Having experienced delays a...,2023-06-22,United States,2023,6,22


In [210]:
df.drop_duplicates('reviews',inplace=True)

In [211]:
df.groupby('year').reviews.value_counts(ascending=False)

year  reviews                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [212]:
#plt.plot(df['year'].value_counts().sort_index(ascending=False))

In [213]:
#plt.plot(df['month'].value_counts().sort_index(ascending=False))

In [214]:
#plt.plot(df['day'].value_counts().sort_index(ascending=False))

In [215]:
#plt.figure(figsize=(20,6))
#plt.xticks(rotation=90)
#sns.countplot(df['place'].sort_index(ascending=False));

In [216]:
def preprocessing(text):
    text = text.lower()

    text = re.sub(r'[^\w\s]','',text)

    text = nltk.word_tokenize(text)

    text = [word for word in text if not word in stopwords.words('english')]

    text = [PorterStemmer().stem(w) for w in text]

    text = [WordNetLemmatizer().lemmatize(w) for w in text]

    text = ' '.join(text)
    return text

In [217]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [218]:
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [219]:
analyzer = SentimentIntensityAnalyzer()

In [220]:
def score(score):
    if score>0.00:
        return 'positive'
    elif score < 0.0:
        return 'negative'
    else:
        return 'neutral'

In [221]:
import copy

In [222]:
d = copy.deepcopy(df)

In [223]:
d.head()

,title,reviews,date,place,year,month,day
0,"""no boarding drinks provided""",✅ Trip Verified | First time flying with BA b...,2023-06-25,United Kingdom,2023,6,25
1,"""WiFi didn't work""",Not Verified | You can buy sandwiches and cris...,2023-06-24,United Kingdom,2023,6,24
2,"""stick with economy""",✅ Trip Verified | This is a two-for-one revie...,2023-06-24,United Kingdom,2023,6,24
3,"""Communication is terrible""",✅ Trip Verified | Absolutely horrible airline....,2023-06-23,United States,2023,6,23
4,"""delays and cancellations""",✅ Trip Verified | Having experienced delays a...,2023-06-22,United States,2023,6,22


In [224]:
d['title'] = d['title'].apply(preprocessing)

In [225]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [226]:
d.head()

,title,reviews,date,place,year,month,day
0,board drink provid,✅ Trip Verified | First time flying with BA b...,2023-06-25,United Kingdom,2023,6,25
1,wifi didnt work,Not Verified | You can buy sandwiches and cris...,2023-06-24,United Kingdom,2023,6,24
2,stick economi,✅ Trip Verified | This is a two-for-one revie...,2023-06-24,United Kingdom,2023,6,24
3,commun terribl,✅ Trip Verified | Absolutely horrible airline....,2023-06-23,United States,2023,6,23
4,delay cancel,✅ Trip Verified | Having experienced delays a...,2023-06-22,United States,2023,6,22


In [227]:
d['reviews'] = d['reviews'].apply(preprocessing)

In [228]:
d['place'] = d['place'].apply(preprocessing)

In [229]:
d.head()

,title,reviews,date,place,year,month,day
0,board drink provid,trip verifi first time fli ba busi class never...,2023-06-25,unit kingdom,2023,6,25
1,wifi didnt work,verifi buy sandwich crisp dont expect hot meal...,2023-06-24,unit kingdom,2023,6,24
2,stick economi,trip verifi twoforon review cover economi busi...,2023-06-24,unit kingdom,2023,6,24
3,commun terribl,trip verifi absolut horribl airlin commun terr...,2023-06-23,unit state,2023,6,23
4,delay cancel,trip verifi experienc delay cancel depart usa ...,2023-06-22,unit state,2023,6,22


In [230]:
revi_score = []
for value in d['reviews'].values:
    revi_score.append(analyzer.polarity_scores(value)['compound'])

d['revi_score'] = revi_score

In [231]:
d.head()

,title,reviews,date,place,year,month,day,revi_score
0,board drink provid,trip verifi first time fli ba busi class never...,2023-06-25,unit kingdom,2023,6,25,0.0000
1,wifi didnt work,verifi buy sandwich crisp dont expect hot meal...,2023-06-24,unit kingdom,2023,6,24,0.3773
2,stick economi,trip verifi twoforon review cover economi busi...,2023-06-24,unit kingdom,2023,6,24,0.9382
3,commun terribl,trip verifi absolut horribl airlin commun terr...,2023-06-23,unit state,2023,6,23,-0.6249
4,delay cancel,trip verifi experienc delay cancel depart usa ...,2023-06-22,unit state,2023,6,22,-0.4767


In [232]:
d['revi_score_sta'] = d['revi_score'].apply(score)

In [233]:
d.head()

,title,reviews,date,place,year,month,day,revi_score,revi_score_sta
0,board drink provid,trip verifi first time fli ba busi class never...,2023-06-25,unit kingdom,2023,6,25,0.0000,neutral
1,wifi didnt work,verifi buy sandwich crisp dont expect hot meal...,2023-06-24,unit kingdom,2023,6,24,0.3773,positive
2,stick economi,trip verifi twoforon review cover economi busi...,2023-06-24,unit kingdom,2023,6,24,0.9382,positive
3,commun terribl,trip verifi absolut horribl airlin commun terr...,2023-06-23,unit state,2023,6,23,-0.6249,negative
4,delay cancel,trip verifi experienc delay cancel depart usa ...,2023-06-22,unit state,2023,6,22,-0.4767,negative


In [234]:
#sns.countplot(d['revi_score_sta'])

In [235]:
#plt.pie(d['revi_score_sta'].value_counts(),labels=['positve','negative','neutral'],autopct='%.2f');

In [236]:
title_score = []
for value in d['title'].values:
    title_score.append(analyzer.polarity_scores(value)['compound'])

d['title_score'] = title_score

In [237]:
d.head()

,title,reviews,date,place,year,month,day,revi_score,revi_score_sta,title_score
0,board drink provid,trip verifi first time fli ba busi class never...,2023-06-25,unit kingdom,2023,6,25,0.0000,neutral,0.0000
1,wifi didnt work,verifi buy sandwich crisp dont expect hot meal...,2023-06-24,unit kingdom,2023,6,24,0.3773,positive,0.0000
2,stick economi,trip verifi twoforon review cover economi busi...,2023-06-24,unit kingdom,2023,6,24,0.9382,positive,0.0000
3,commun terribl,trip verifi absolut horribl airlin commun terr...,2023-06-23,unit state,2023,6,23,-0.6249,negative,0.0000
4,delay cancel,trip verifi experienc delay cancel depart usa ...,2023-06-22,unit state,2023,6,22,-0.4767,negative,-0.5106


In [238]:
d['title_score_sta'] = d['title_score'].apply(score)

In [239]:
d.head()

,title,reviews,date,place,year,month,day,revi_score,revi_score_sta,title_score,title_score_sta
0,board drink provid,trip verifi first time fli ba busi class never...,2023-06-25,unit kingdom,2023,6,25,0.0000,neutral,0.0000,neutral
1,wifi didnt work,verifi buy sandwich crisp dont expect hot meal...,2023-06-24,unit kingdom,2023,6,24,0.3773,positive,0.0000,neutral
2,stick economi,trip verifi twoforon review cover economi busi...,2023-06-24,unit kingdom,2023,6,24,0.9382,positive,0.0000,neutral
3,commun terribl,trip verifi absolut horribl airlin commun terr...,2023-06-23,unit state,2023,6,23,-0.6249,negative,0.0000,neutral
4,delay cancel,trip verifi experienc delay cancel depart usa ...,2023-06-22,unit state,2023,6,22,-0.4767,negative,-0.5106,negative


In [240]:
#sns.countplot(d['title_score_sta'])

In [241]:
#plt.pie(d['title_score_sta'].value_counts(),labels=['neutral','positive','negative'],autopct='%.2f');

In [242]:
d.sort_values(by='date',inplace=True,ascending=False)

In [243]:
d.head()

,title,reviews,date,place,year,month,day,revi_score,revi_score_sta,title_score,title_score_sta
0,board drink provid,trip verifi first time fli ba busi class never...,2023-06-25,unit kingdom,2023,6,25,0.0000,neutral,0.0000,neutral
2,stick economi,trip verifi twoforon review cover economi busi...,2023-06-24,unit kingdom,2023,6,24,0.9382,positive,0.0000,neutral
1,wifi didnt work,verifi buy sandwich crisp dont expect hot meal...,2023-06-24,unit kingdom,2023,6,24,0.3773,positive,0.0000,neutral
3,commun terribl,trip verifi absolut horribl airlin commun terr...,2023-06-23,unit state,2023,6,23,-0.6249,negative,0.0000,neutral
4,delay cancel,trip verifi experienc delay cancel depart usa ...,2023-06-22,unit state,2023,6,22,-0.4767,negative,-0.5106,negative


In [244]:
d.groupby('year').revi_score.mean().sort_values(ascending=False)

year
2015    0.390824
2016    0.353458
2019    0.273862
2018    0.270908
2020    0.249677
2017    0.219365
2021    0.208007
2022    0.179797
2023    0.002803
Name: revi_score, dtype: float64

In [245]:
d2015 = d[d['year']==2015]

In [246]:
score2015 = d2015['revi_score'].mean()
score2015

0.3908238434163701

In [247]:
sum(d['revi_score'])/len(d)

0.2786858022579543

In [248]:
l = [2015,
2016,
2019,
2018,
2020,
2017,
2021,
2022,
2023]

l = sorted(l)
sc = []
for i in l:
    su = d[d['year']==i]
    sc.append(sum(su['revi_score'])/len(su))

In [249]:
sc

[0.39082384341637044,
 0.3534584210526315,
 0.2193652097902099,
 0.27090833333333314,
 0.2738620786516852,
 0.24967692307692305,
 0.20800689655172408,
 0.17979718309859147,
 0.002802962962962943]

In [250]:
d.groupby('year').title_score.mean().sort_values(ascending=False)

year
2021    0.056510
2022    0.055426
2019    0.053767
2016    0.031614
2018    0.026934
2015    0.021610
2020    0.020567
2017   -0.018506
2023   -0.050856
Name: title_score, dtype: float64

In [251]:
#plt.plot(d.groupby('year').revi_score.mean(),label='review score')
#plt.plot(d.groupby('year').title_score.mean(),label='title score')
#plt.legend()
#plt.show()

In [252]:
#plt.figure(figsize=(12,4))
#plt.plot(d.groupby('month').revi_score.mean(),label='revi score')
#plt.plot(d.groupby('month').title_score.mean(),label='title score')
#plt.legend()
#plt.show()

In [253]:
#plt.figure(figsize=(12,4))
#plt.plot(d.groupby('day').revi_score.mean(),label='revi score')
#plt.plot(d.groupby('day').title_score.mean(),label='title score')
#plt.legend()
#plt.show()

In [254]:
#plt.figure(figsize=(12,4))
#plt.plot(d['revi_score'])

In [255]:
#plt.scatter(d['day'],d['revi_score'])

In [256]:
score = (d['revi_score']-d['revi_score'].min())/(d['revi_score'].max()-d['revi_score'].min())

In [257]:
#plt.plot(score)

In [258]:
#plt.figure(figsize=(12,4))
#sns.countplot(d.revi_score.value_counts());

In [259]:
d.groupby('year').revi_score.value_counts().sort_values(ascending=False)

year  revi_score
2019   0.3612       9
2017   0.7269       7
       0.0000       7
      -0.4215       7
2016   0.5859       7
                   ..
2017  -0.6426       1
      -0.6470       1
      -0.6491       1
      -0.6597       1
2023   0.9933       1
Name: revi_score, Length: 1911, dtype: int64

In [260]:
#fig = go.Figure(data=go.Mesh3d(x = d['year'],
#                              y = d['revi_score'],
#                              z = d['title_score']))
#fig

In [261]:
#fig = px.line(d['revi_score'])
#fig

In [262]:
#fig = px.line_3d(d, x='revi_score',
#                y='year',
#                z='year')
#fig

In [263]:
#import plotly.express as px
#fig = px.scatter(d,'year',
#                              #y = 'title_score',
#                               'revi_score',color='revi_score')
#fig.show()

In [264]:
#fig = px.scatter(d, 'year', 'title_score',color='title_score')
#fig.show()

In [265]:
d.groupby('place').revi_score.mean().sort_values()

place
indonesia    -0.639300
egypt        -0.493900
nigeria      -0.442150
bermuda      -0.425450
taiwan       -0.308533
                ...   
luxembourg    0.905850
brazil        0.919750
panama        0.946000
vietnam       0.983300
iceland       0.983700
Name: revi_score, Length: 67, dtype: float64

In [266]:
#plt.figure(figsize=(12,4))
#plt.xticks(rotation=90)
#sns.barplot(d['place'],d['revi_score'].sort_values(ascending=False));

In [267]:
#plt.figure(figsize=(12,4))
#text = ' '.join(d['reviews'])
#wordcloud = WordCloud().generate(text)
#plt.imshow(wordcloud, interpolation='bilinear')
#plt.axis('off')
#plt.show()

## Data Modeling

In [268]:
#text = list(d['reviews'])
#docs = [simple_preprocess(doc) for doc in text]
#dictionary = corpora.Dictionary(docs)
#corpus = [dictionary.doc2bow(doc) for doc in docs]
#num = 10
#lda_model = gensim.models.LdaModel(corpus, num_topics=num,id2word=dictionary, passes=10)
#for topic_id, topic_word in lda_model.print_topics():
#    print(f'topic {topic_id}: {topic_word} \n')

In [269]:
#text = list(d['title'])
#docs = [simple_preprocess(doc) for doc in text]
#dictionary = corpora.Dictionary(docs)
#corpus = [dictionary.doc2bow(doc) for doc in docs]
#num = 10
#lda_model = gensim.models.LdaModel(corpus, num_topics=num,id2word=dictionary, passes=10)
#for topic_id, topic_word in lda_model.print_topics():
#    print(f'topic {topic_id}: {topic_word} \n')

## Text Classification

In [270]:
d.head()

,title,reviews,date,place,year,month,day,revi_score,revi_score_sta,title_score,title_score_sta
0,board drink provid,trip verifi first time fli ba busi class never...,2023-06-25,unit kingdom,2023,6,25,0.0000,neutral,0.0000,neutral
2,stick economi,trip verifi twoforon review cover economi busi...,2023-06-24,unit kingdom,2023,6,24,0.9382,positive,0.0000,neutral
1,wifi didnt work,verifi buy sandwich crisp dont expect hot meal...,2023-06-24,unit kingdom,2023,6,24,0.3773,positive,0.0000,neutral
3,commun terribl,trip verifi absolut horribl airlin commun terr...,2023-06-23,unit state,2023,6,23,-0.6249,negative,0.0000,neutral
4,delay cancel,trip verifi experienc delay cancel depart usa ...,2023-06-22,unit state,2023,6,22,-0.4767,negative,-0.5106,negative


In [271]:
from sklearn.preprocessing import LabelEncoder

In [272]:
labels = d['revi_score_sta']
feature = d['reviews']

In [273]:
le = LabelEncoder()
labels = le.fit_transform(labels)

In [274]:
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(feature)

In [275]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features,labels,test_size=0.2)

In [276]:
from sklearn.svm import SVC
m = SVC()
m = m.fit(x_train, y_train)
pred = m.predict(x_test)

In [277]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test, pred))

0.7914529914529914


In [278]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.76      0.63      0.69       202
           1       0.00      0.00      0.00         9
           2       0.80      0.90      0.85       374

    accuracy                           0.79       585
   macro avg       0.52      0.51      0.51       585
weighted avg       0.78      0.79      0.78       585



In [279]:
labels = d['title_score_sta']
feature = d['title']
le = LabelEncoder()
labels = le.fit_transform(labels)
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(feature)
x_train, x_test, y_train, y_test = train_test_split(features,labels,test_size=0.2)
m = SVC()
m = m.fit(x_train, y_train)
pred = m.predict(x_test)

In [280]:
print(accuracy_score(y_test, pred))

0.8905982905982905


In [281]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.91      0.72      0.80       103
           1       0.87      1.00      0.93       337
           2       0.96      0.77      0.85       145

    accuracy                           0.89       585
   macro avg       0.91      0.83      0.86       585
weighted avg       0.90      0.89      0.89       585



In [282]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
m = rf.fit(x_train,y_train)
pred = m.predict(x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.91      0.81      0.86       103
           1       0.92      0.99      0.95       337
           2       0.97      0.86      0.91       145

    accuracy                           0.93       585
   macro avg       0.93      0.89      0.91       585
weighted avg       0.93      0.93      0.93       585



In [283]:
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
m = nb.fit(x_train, y_train)
pred = m.predict(x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.81      0.68      0.74       103
           1       0.88      0.95      0.92       337
           2       0.86      0.81      0.84       145

    accuracy                           0.87       585
   macro avg       0.85      0.81      0.83       585
weighted avg       0.87      0.87      0.87       585



In [284]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential

In [285]:
tf.random.set_seed(100)
model = Sequential()
model.add(Dense(256, activation='relu',input_dim=(1481)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [286]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [287]:
fit = model.fit(x_train, y_train, epochs=10,validation_data=(x_test, y_test))

Epoch 1/10
74/74 [==============================] - 2s 11ms/step - loss: 0.0000e+00 - accuracy: 0.5701 - val_loss: 0.0000e+00 - val_accuracy: 0.5145
Epoch 2/10
74/74 [==============================] - 1s 8ms/step - loss: 0.0000e+00 - accuracy: 0.5269 - val_loss: 0.0000e+00 - val_accuracy: 0.4974
Epoch 3/10
74/74 [==============================] - 1s 9ms/step - loss: 0.0000e+00 - accuracy: 0.5749 - val_loss: 0.0000e+00 - val_accuracy: 0.5333
Epoch 4/10
74/74 [==============================] - 1s 9ms/step - loss: 0.0000e+00 - accuracy: 0.5932 - val_loss: 0.0000e+00 - val_accuracy: 0.5350
Epoch 5/10
74/74 [==============================] - 1s 9ms/step - loss: 0.0000e+00 - accuracy: 0.5843 - val_loss: 0.0000e+00 - val_accuracy: 0.5436
Epoch 6/10
74/74 [==============================] - 1s 13ms/step - loss: 0.0000e+00 - accuracy: 0.6095 - val_loss: 0.0000e+00 - val_accuracy: 0.5487
Epoch 7/10
74/74 [==============================] - 1s 14ms/step - loss: 0.0000e+00 - accuracy: 0.6082 - val_l

In [294]:
import pandas as pd
import spacy

ner_list = []
nlp = spacy.load('en_core_web_sm')

# Assuming your DataFrame is called 'df' and the column with reviews is named 'reviews'
for review in d['reviews']:
    doc = nlp(review)
    # Perform further processing or analysis with the 'doc' object
    # For example, you can access entities in the document using 'doc.ents'
    for entity in doc.ents:
        ner_list.append({'Entity': entity.text, 'Label': entity.label_})

# Create the DataFrame from the list of entities
ner = pd.DataFrame(ner_list)

# Print the DataFrame
print(ner)


                 Entity     Label
0                 first   ORDINAL
1               qualiti    PERSON
2      standard qualiti    PERSON
3                  chao    PERSON
4                london       GPE
...                 ...       ...
25676               25a  CARDINAL
25677               one  CARDINAL
25678               one  CARDINAL
25679          european      NORP
25680             930am       GPE

[25681 rows x 2 columns]


In [295]:
ner

,Entity,Label
0,first,ORDINAL
1,qualiti,PERSON
2,standard qualiti,PERSON
3,chao,PERSON
4,london,GPE
...,...,...
25676,25a,CARDINAL
25677,one,CARDINAL
25678,one,CARDINAL
25679,european,NORP


In [296]:
ner.groupby('Entity').Label.value_counts().sort_values(ascending=False)[:30]

Entity    Label   
british   NORP        1851
london    GPE         1771
verifi    ORG         1532
first     ORDINAL      904
one       CARDINAL     844
two       CARDINAL     343
passeng   PERSON       277
airlin    PERSON       243
2         CARDINAL     239
3         CARDINAL     212
airlin    GPE          192
5         CARDINAL     185
second    ORDINAL      169
experi    NORP         166
30        CARDINAL     147
departur  GPE          138
20        CARDINAL     134
issu      ORG          132
2 hour    TIME         119
747       PRODUCT      119
777       CARDINAL     117
hour      TIME         116
4         CARDINAL     114
chang     PERSON       113
three     CARDINAL     110
10        CARDINAL     106
half      CARDINAL     103
1         CARDINAL      98
3 hour    TIME          94
plu       ORG           93
Name: Label, dtype: int64

In [299]:
ner.groupby('Label').Entity.value_counts().sort_values(ascending=False)[:30]

Label     Entity  
NORP      british     1851
GPE       london      1771
ORG       verifi      1532
ORDINAL   first        904
CARDINAL  one          844
          two          343
PERSON    passeng      277
          airlin       243
CARDINAL  2            239
          3            212
GPE       airlin       192
CARDINAL  5            185
ORDINAL   second       169
NORP      experi       166
CARDINAL  30           147
GPE       departur     138
CARDINAL  20           134
ORG       issu         132
TIME      2 hour       119
PRODUCT   747          119
CARDINAL  777          117
TIME      hour         116
CARDINAL  4            114
PERSON    chang        113
CARDINAL  three        110
          10           106
          half         103
          1             98
TIME      3 hour        94
ORG       plu           93
Name: Entity, dtype: int64

In [300]:
import pandas as pd
import spacy

ner_list = []
nlp = spacy.load('en_core_web_sm')

# Assuming your DataFrame is called 'df' and the column with reviews is named 'reviews'
for review in df['reviews']:
    doc = nlp(review)
    # Perform further processing or analysis with the 'doc' object
    # For example, you can access entities in the document using 'doc.ents'
    for entity in doc.ents:
        ner_list.append({'Entity': entity.text, 'Label': entity.label_})

# Create the DataFrame from the list of entities
ner = pd.DataFrame(ner_list)

# Print the DataFrame
print(ner)


              Entity     Label
0      Trip Verified    PERSON
1              First   ORDINAL
2                 BA      NORP
3             Bodrum    PERSON
4                 BA       ORG
...              ...       ...
31975            one  CARDINAL
31976             11  CARDINAL
31977            25A      DATE
31978            one  CARDINAL
31979       European      NORP

[31980 rows x 2 columns]


In [301]:
ner.groupby('Entity').Label.value_counts().sort_values(ascending=False)[:30]

Entity           Label   
BA               ORG         2451
British Airways  ORG         1692
London           GPE         1401
Trip Verified    PERSON      1065
Verified Review  ORG          769
BA               NORP         727
first            ORDINAL      597
Heathrow         FAC          548
one              CARDINAL     435
London Heathrow  PERSON       370
two              CARDINAL     337
First            ORDINAL      337
LHR              ORG          299
2                CARDINAL     258
3                CARDINAL     203
Boeing           ORG          200
Cabin            PERSON       200
A380             PRODUCT      166
second           ORDINAL      155
Club World       ORG          151
Club Europe      LOC          149
Gatwick          GPE          139
Singapore        GPE          135
747              PRODUCT      122
4                CARDINAL     116
UK               GPE          107
One              CARDINAL     106
Ryanair          PERSON       105
British          NORP 

In [302]:
ner.groupby('Label').Entity.value_counts().sort_values(ascending=False)[:30]

Label     Entity         
ORG       BA                 2451
          British Airways    1692
GPE       London             1401
PERSON    Trip Verified      1065
ORG       Verified Review     769
NORP      BA                  727
ORDINAL   first               597
FAC       Heathrow            548
CARDINAL  one                 435
PERSON    London Heathrow     370
ORDINAL   First               337
CARDINAL  two                 337
ORG       LHR                 299
CARDINAL  2                   258
          3                   203
ORG       Boeing              200
PERSON    Cabin               200
PRODUCT   A380                166
ORDINAL   second              155
ORG       Club World          151
LOC       Club Europe         149
GPE       Gatwick             139
          Singapore           135
PRODUCT   747                 122
CARDINAL  4                   116
GPE       UK                  107
CARDINAL  One                 106
PERSON    Ryanair             105
NORP      British     